In [1]:
# ---------------------------------
# Cell 1: Install & Import Libraries
# ---------------------------------

!pip install opendatasets --quiet

import opendatasets as od
import os
import tensorflow as tf
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import numpy as np
import time
import warnings

# Suppress warnings
warnings.filterwarnings('ignore')

In [2]:
# ---------------------------------
# Cell 1.5: Mount Google Drive
# ---------------------------------
from google.colab import drive
drive.mount('/content/drive')

print("Google Drive mounted!")

Mounted at /content/drive
Google Drive mounted!


In [3]:
# -------------------------------------------
# Cell 2: Download Dataset & Define Constants
# -------------------------------------------

# Download the dataset
# This will prompt for your Kaggle username and API key
print("Please enter your Kaggle credentials:")
dataset_url = 'https://www.kaggle.com/datasets/almightyj/person-face-sketches'
od.download(dataset_url)

# Define paths and constants
DATA_DIR = './person-face-sketches/'
PHOTO_DIR = os.path.join(DATA_DIR, 'photos')
SKETCH_DIR = os.path.join(DATA_DIR, 'sketches')

# Model parameters
BUFFER_SIZE = 1000  # For shuffling
BATCH_SIZE = 1      # As per the CycleGAN paper
IMG_WIDTH = 256
IMG_HEIGHT = 256
OUTPUT_CHANNELS = 3
LAMBDA_CYCLE = 10.0
LAMBDA_IDENTITY = 0.5 * LAMBDA_CYCLE

Please enter your Kaggle credentials:
Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: munjenko
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/almightyj/person-face-sketches


100%|██████████| 1.29G/1.29G [00:15<00:00, 90.8MB/s]

In [4]:
# -----------------------------------------------
# Cell 2.5 (Final Version): Set Correct Paths and Verify
# -----------------------------------------------
import os

# This is the main folder you downloaded
DATA_DIR = './person-face-sketches/'

# --- THIS IS THE FIX ---
# The actual image folders are inside the 'train' directory.
PHOTO_DIR = os.path.join(DATA_DIR, 'train', 'photos')
SKETCH_DIR = os.path.join(DATA_DIR, 'train', 'sketches')
# ---------------------

print(f"Corrected PHOTO_DIR path: {PHOTO_DIR}")
print(f"Corrected SKETCH_DIR path: {SKETCH_DIR}")

print("\n--- Verifying Corrected Paths ---")

if os.path.exists(PHOTO_DIR) and os.path.exists(SKETCH_DIR):
    # Count files in the correct 'photos' directory
    !echo "Counting photos in 'train/photos'..."
    !find {PHOTO_DIR} -type f -name "*.jpg" | wc -l

    # Count files in the correct 'sketches' directory
    !echo "Counting sketches in 'train/sketches'..."
    !find {SKETCH_DIR} -type f -name "*.jpg" | wc -l

    print("\nVerification complete. You should now see large file counts (e.g., 4000+) for both.")

else:
    print(f"ERROR: Still cannot find directories.")
    print(f"Checked for photos at: {PHOTO_DIR}")
    print(f"Checked for sketches at: {SKETCH_DIR}")
    print("Please double-check the folder names.")

Corrected PHOTO_DIR path: ./person-face-sketches/train/photos
Corrected SKETCH_DIR path: ./person-face-sketches/train/sketches

--- Verifying Corrected Paths ---
Counting photos in 'train/photos'...
20655
Counting sketches in 'train/sketches'...
20655

Verification complete. You should now see large file counts (e.g., 4000+) for both.


In [5]:
# -----------------------------------
# Cell 3: Preprocessing Functions
# -----------------------------------

def load_image(image_file):
    image = tf.io.read_file(image_file)
    image = tf.image.decode_jpeg(image, channels=3)
    return image

def resize(image, height, width):
    image = tf.image.resize(image, [height, width],
                            method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    return image

# Normalize images to [-1, 1]
def normalize(image):
    image = tf.cast(image, tf.float32)
    image = (image / 127.5) - 1
    return image

# --- Load and Preprocess for Training ---
def load_image_train(image_file):
    image = load_image(image_file)
    # Simple data augmentation: random flipping
    image = tf.image.random_flip_left_right(image)
    image = resize(image, IMG_HEIGHT, IMG_WIDTH)
    image = normalize(image)
    return image

# --- Load and Preprocess for Testing (if needed) ---
def load_image_test(image_file):
    image = load_image(image_file)
    image = resize(image, IMG_HEIGHT, IMG_WIDTH)
    image = normalize(image)
    return image

In [6]:
# ---------------------------------------------
# Cell 4 (New Version): Create Half-Size Dataset
# ---------------------------------------------

print("Creating training datasets...")

# Create datasets by listing all files
train_photos_full = tf.data.Dataset.list_files(PHOTO_DIR + '/*.jpg', shuffle=False)
train_sketches_full = tf.data.Dataset.list_files(SKETCH_DIR + '/*.jpg', shuffle=False)

# --- THIS IS THE CHANGE ---
# Get the size (from your verification step) and calculate half
dataset_size = 4000
half_size = 4000

print(f"Original dataset size found: {dataset_size}")
print(f"Using new half-size: {half_size}")

# Take only the first half of the datasets
train_photos = train_photos_full.take(half_size)
train_sketches = train_sketches_full.take(half_size)
# --------------------------

# Now, process (map) only the half-size datasets
train_photos = train_photos.map(load_image_train,
                                num_parallel_calls=tf.data.AUTOTUNE)
train_photos = train_photos.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

train_sketches = train_sketches.map(load_image_train,
                                   num_parallel_calls=tf.data.AUTOTUNE)
train_sketches = train_sketches.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# Zip the two datasets to train them together
train_dataset = tf.data.Dataset.zip((train_photos, train_sketches))

print("\nHalf-size datasets created successfully.")
print(f"New train dataset: {train_dataset}")

Creating training datasets...
Original dataset size found: 4000
Using new half-size: 4000

Half-size datasets created successfully.
New train dataset: <_ZipDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None))>


In [7]:
# -----------------------------------------
# Cell 5: Model Architecture (Corrected)
# -----------------------------------------

# FIX: Replace InstanceNormalization with GroupNormalization
def instance_norm():
    """Returns a GroupNormalization layer functioning as InstanceNormalization."""
    return layers.GroupNormalization(groups=-1, epsilon=1e-5)

def downsample(filters, size, apply_norm=True):
    initializer = tf.random_normal_initializer(0., 0.02)
    result = tf.keras.Sequential()
    result.add(layers.Conv2D(filters, size, strides=2, padding='same',
                             kernel_initializer=initializer, use_bias=False))
    if apply_norm:
        # --- THIS LINE IS CHANGED ---
        result.add(instance_norm())
    result.add(layers.LeakyReLU())
    return result

def upsample(filters, size, apply_dropout=False):
    initializer = tf.random_normal_initializer(0., 0.02)
    result = tf.keras.Sequential()
    result.add(layers.Conv2DTranspose(filters, size, strides=2, padding='same',
                                     kernel_initializer=initializer, use_bias=False))
    # --- THIS LINE IS CHANGED ---
    result.add(instance_norm())
    if apply_dropout:
        result.add(layers.Dropout(0.5))
    result.add(layers.ReLU())
    return result

def build_generator():
    inputs = layers.Input(shape=[IMG_WIDTH, IMG_HEIGHT, 3])

    # Downsampling
    down_stack = [
        downsample(64, 4, apply_norm=False), # (bs, 128, 128, 64)
        downsample(128, 4),                 # (bs, 64, 64, 128)
        downsample(256, 4),                 # (bs, 32, 32, 256)
        downsample(512, 4),                 # (bs, 16, 16, 512)
        downsample(512, 4),                 # (bs, 8, 8, 512)
        downsample(512, 4),                 # (bs, 4, 4, 512)
        downsample(512, 4),                 # (bs, 2, 2, 512)
        downsample(512, 4),                 # (bs, 1, 1, 512)
    ]

    # Upsampling
    up_stack = [
        upsample(512, 4, apply_dropout=True), # (bs, 2, 2, 1024)
        upsample(512, 4, apply_dropout=True), # (bs, 4, 4, 1024)
        upsample(512, 4, apply_dropout=True), # (bs, 8, 8, 1024)
        upsample(512, 4),                     # (bs, 16, 16, 1024)
        upsample(256, 4),                     # (bs, 32, 32, 512)
        upsample(128, 4),                     # (bs, 64, 64, 256)
        upsample(64, 4),                      # (bs, 128, 128, 128)
    ]

    initializer = tf.random_normal_initializer(0., 0.02)
    last = layers.Conv2DTranspose(OUTPUT_CHANNELS, 4,
                                  strides=2,
                                  padding='same',
                                  kernel_initializer=initializer,
                                  activation='tanh') # Output is [-1, 1]

    x = inputs
    skips = []
    for down in down_stack:
        x = down(x)
        skips.append(x)

    skips = reversed(skips[:-1])

    for up, skip in zip(up_stack, skips):
        x = up(x)
        x = layers.Concatenate()([x, skip])

    x = last(x)

    return tf.keras.Model(inputs=inputs, outputs=x)

def build_discriminator():
    initializer = tf.random_normal_initializer(0., 0.02)
    inp = layers.Input(shape=[IMG_WIDTH, IMG_HEIGHT, 3], name='input_image')

    down1 = downsample(64, 4, False)(inp) # (bs, 128, 128, 64)
    down2 = downsample(128, 4)(down1)     # (bs, 64, 64, 128)
    down3 = downsample(256, 4)(down2)     # (bs, 32, 32, 256)

    zero_pad1 = layers.ZeroPadding2D()(down3) # (bs, 34, 34, 256)
    conv = layers.Conv2D(512, 4, strides=1,
                         kernel_initializer=initializer,
                         use_bias=False)(zero_pad1) # (bs, 31, 31, 512)

    # --- THIS LINE IS CHANGED ---
    norm1 = instance_norm()(conv)
    leaky_relu = layers.LeakyReLU()(norm1)
    zero_pad2 = layers.ZeroPadding2D()(leaky_relu) # (bs, 33, 33, 512)

    last = layers.Conv2D(1, 4, strides=1,
                         kernel_initializer=initializer)(zero_pad2) # (bs, 30, 30, 1)

    return tf.keras.Model(inputs=inp, outputs=last)

print("Model architecture defined.")

Model architecture defined.


In [8]:
# -----------------------------------------------
# Cell 6: Build Models, Losses, & Optimizers
# -----------------------------------------------

# Instantiate the models
gen_G = build_generator() # Photo -> Sketch
gen_F = build_generator() # Sketch -> Photo

disc_X = build_discriminator() # Discriminates Photos (Domain A)
disc_Y = build_discriminator() # Discriminates Sketches (Domain B)

print("Models instantiated.")

# --- Loss Functions ---
# We use Mean Squared Error for LSGAN loss, which is more stable
loss_obj = tf.keras.losses.MeanSquaredError()
mae_loss = tf.keras.losses.MeanAbsoluteError()

def discriminator_loss(real, generated):
    real_loss = loss_obj(tf.ones_like(real), real)
    generated_loss = loss_obj(tf.zeros_like(generated), generated)
    total_disc_loss = (real_loss + generated_loss) * 0.5
    return total_disc_loss

def generator_loss(generated):
    # The generator wants the discriminator to think the fake image is real
    return loss_obj(tf.ones_like(generated), generated)

def cycle_loss(real_image, cycled_image):
    loss = mae_loss(real_image, cycled_image)
    return LAMBDA_CYCLE * loss

def identity_loss(real_image, same_image):
    loss = mae_loss(real_image, same_image)
    return LAMBDA_IDENTITY * loss

# --- Optimizers ---
generator_g_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
generator_f_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
discriminator_x_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
discriminator_y_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

print("Losses and optimizers defined.")

Models instantiated.
Losses and optimizers defined.


In [9]:
# ---------------------------------
# Cell 7 (New Version): Checkpoint Setup with Google Drive
# ---------------------------------

# --- THIS IS THE CHANGE ---
# We point the path to a folder in your Google Drive
checkpoint_path = "/content/drive/MyDrive/Colab_CheckPoints/CycleGAN/train"

# Create the directory if it doesn't exist
!mkdir -p {checkpoint_path}
# --------------------------

ckpt = tf.train.Checkpoint(generator_g=gen_G,
                           generator_f=gen_F,
                           discriminator_x=disc_X,
                           discriminator_y=disc_Y,
                           generator_g_optimizer=generator_g_optimizer,
                           generator_f_optimizer=generator_f_optimizer,
                           discriminator_x_optimizer=discriminator_x_optimizer,
                           discriminator_y_optimizer=discriminator_y_optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

# Restore from last checkpoint if it exists
if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print (f'Latest checkpoint restored from {checkpoint_path}!')
else:
    print('No checkpoint found. Starting from scratch.')

Latest checkpoint restored from /content/drive/MyDrive/Colab_CheckPoints/CycleGAN/train!


In [10]:
# -----------------------------------
# Cell 8: The train_step Function
# -----------------------------------

@tf.function
def train_step(real_photo, real_sketch):
    # persistent=True allows us to calculate multiple gradients
    with tf.GradientTape(persistent=True) as tape:

        # --- Forward Pass ---
        fake_sketch = gen_G(real_photo, training=True)
        fake_photo = gen_F(real_sketch, training=True)

        cycled_photo = gen_F(fake_sketch, training=True)
        cycled_sketch = gen_G(fake_photo, training=True)

        same_photo = gen_F(real_photo, training=True)
        same_sketch = gen_G(real_sketch, training=True)

        disc_real_photo = disc_X(real_photo, training=True)
        disc_fake_photo = disc_X(fake_photo, training=True)

        disc_real_sketch = disc_Y(real_sketch, training=True)
        disc_fake_sketch = disc_Y(fake_sketch, training=True)

        # --- Calculate Losses ---
        # 1. Generator losses
        gen_g_loss = generator_loss(disc_fake_sketch) # G wants to fool D_Y
        gen_f_loss = generator_loss(disc_fake_photo) # F wants to fool D_X

        # 2. Cycle-consistency losses
        total_cycle_loss = cycle_loss(real_photo, cycled_photo) + cycle_loss(real_sketch, cycled_sketch)

        # 3. Identity losses
        total_identity_loss = identity_loss(real_photo, same_photo) + identity_loss(real_sketch, same_sketch)

        # Total Generator Loss
        total_gen_g_loss = gen_g_loss + total_cycle_loss + total_identity_loss
        total_gen_f_loss = gen_f_loss + total_cycle_loss + total_identity_loss

        # 4. Discriminator losses
        disc_x_loss = discriminator_loss(disc_real_photo, disc_fake_photo)
        disc_y_loss = discriminator_loss(disc_real_sketch, disc_fake_sketch)

    # --- Calculate and Apply Gradients ---
    generator_g_gradients = tape.gradient(total_gen_g_loss, gen_G.trainable_variables)
    generator_f_gradients = tape.gradient(total_gen_f_loss, gen_F.trainable_variables)

    discriminator_x_gradients = tape.gradient(disc_x_loss, disc_X.trainable_variables)
    discriminator_y_gradients = tape.gradient(disc_y_loss, disc_Y.trainable_variables)

    generator_g_optimizer.apply_gradients(zip(generator_g_gradients, gen_G.trainable_variables))
    generator_f_optimizer.apply_gradients(zip(generator_f_gradients, gen_F.trainable_variables))

    discriminator_x_optimizer.apply_gradients(zip(discriminator_x_gradients, disc_X.trainable_variables))
    discriminator_y_optimizer.apply_gradients(zip(discriminator_y_gradients, disc_Y.trainable_variables))

    # --- THIS IS THE PART THAT WAS LIKELY MISSING ---
    return {
        "gen_g_loss": total_gen_g_loss,
        "gen_f_loss": total_gen_f_loss,
        "disc_x_loss": disc_x_loss,
        "disc_y_loss": disc_y_loss,
        "cycle_loss": total_cycle_loss,
        "identity_loss": total_identity_loss
    }

print("Train step function (re)defined.")

Train step function (re)defined.


In [11]:
# -----------------------------
# Cell 9: The Training Loop
# -----------------------------

EPOCHS = 3 # You can start with 10-20, then increase

print(f"Starting training for {EPOCHS} epochs...")

for epoch in range(EPOCHS):
    start = time.time()
    n = 0
    total_g_loss = 0
    total_d_loss = 0

    for (photo, sketch) in train_dataset:
        losses = train_step(photo, sketch)
        if n % 200 == 0:
            print(f"  Batch {n}: G_loss={losses['gen_g_loss']:.4f}, D_X_loss={losses['disc_x_loss']:.4f}")

        total_g_loss += losses['gen_g_loss']
        total_d_loss += (losses['disc_x_loss'] + losses['disc_y_loss']) / 2
        n += 1

    # Save a checkpoint every epoch
    ckpt_save_path = ckpt_manager.save()
    print(f'Saving checkpoint for epoch {epoch+1} at {ckpt_save_path}')

    avg_g_loss = total_g_loss / n
    avg_d_loss = total_d_loss / n

    print(f'Time for epoch {epoch + 1} is {time.time()-start:.2f} sec')
    print(f'  Average G_Loss: {avg_g_loss:.4f}, Average D_Loss: {avg_d_loss:.4f}\n')

print("Training finished.")

Starting training for 3 epochs...
  Batch 0: G_loss=2.1811, D_X_loss=0.0278
  Batch 200: G_loss=2.2780, D_X_loss=0.0604
  Batch 400: G_loss=3.3077, D_X_loss=0.1388
  Batch 600: G_loss=2.2058, D_X_loss=0.3100
  Batch 800: G_loss=2.4485, D_X_loss=0.0781
  Batch 1000: G_loss=2.0018, D_X_loss=0.1721
  Batch 1200: G_loss=2.0849, D_X_loss=0.0680
  Batch 1400: G_loss=2.4529, D_X_loss=0.2373
  Batch 1600: G_loss=2.8073, D_X_loss=0.1018
  Batch 1800: G_loss=3.7866, D_X_loss=0.1597
  Batch 2000: G_loss=2.1221, D_X_loss=0.2265
  Batch 2200: G_loss=1.5943, D_X_loss=0.1148
  Batch 2400: G_loss=2.5869, D_X_loss=0.3096
  Batch 2600: G_loss=1.9985, D_X_loss=0.0639
  Batch 2800: G_loss=2.5654, D_X_loss=0.1235
  Batch 3000: G_loss=1.5866, D_X_loss=0.1247
  Batch 3200: G_loss=2.1496, D_X_loss=0.0786
  Batch 3400: G_loss=1.9774, D_X_loss=0.0400
  Batch 3600: G_loss=1.9483, D_X_loss=0.1340
  Batch 3800: G_loss=2.2799, D_X_loss=0.1542
Saving checkpoint for epoch 1 at /content/drive/MyDrive/Colab_CheckPoints

In [12]:
# ---------------------------------
# Cell 10: Save Final Models
# ---------------------------------

print("Saving final generator models...")

# Save Photo-to-Sketch generator
gen_G.save('photo_to_sketch_generator.h5')

# Save Sketch-to-Photo generator
gen_F.save('sketch_to_photo_generator.h5')

print("Models saved as photo_to_sketch_generator.h5 and sketch_to_photo_generator.h5")

Saving final generator models...


Models saved as photo_to_sketch_generator.h5 and sketch_to_photo_generator.h5


In [16]:
# ---------------------------------
# Cell 11: Write Flask App (app.py)
# ---------------------------------

%%writefile app.py

import flask
from flask import Flask, request, render_template, jsonify
import tensorflow as tf
import numpy as np
import cv2
import base64
import io
from PIL import Image
import warnings

# Suppress TensorFlow warnings
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

app = Flask(__name__)

# --- Model Loading ---
# We no longer need custom_objects because GroupNormalization is standard
print("Loading models...")
try:
    gen_PhotoToSketch = tf.keras.models.load_model('photo_to_sketch_generator.h5', compile=False)
    gen_SketchToPhoto = tf.keras.models.load_model('sketch_to_photo_generator.h5', compile=False)
    print("Models loaded successfully")
except Exception as e:
    print(f"Error loading models: {e}")
    # Create dummy functions if models aren't found
    gen_PhotoToSketch = lambda x: x
    gen_SketchToPhoto = lambda x: x

# --- Helper Functions ---
IMG_WIDTH = 256
IMG_HEIGHT = 256

def preprocess_image(img_bytes):
    # Decode image
    img = Image.open(io.BytesIO(img_bytes)).convert('RGB')
    img_np = np.array(img)

    # Auto-detect logic
    is_sketch = is_image_a_sketch(img_np)

    # Preprocess for model
    img_tensor = tf.convert_to_tensor(img_np, dtype=tf.float32)
    img_tensor = tf.image.resize(img_tensor, [IMG_HEIGHT, IMG_WIDTH])
    img_tensor = (img_tensor / 127.5) - 1 # Normalize to [-1, 1]
    img_tensor = tf.expand_dims(img_tensor, 0) # Add batch dimension

    return img_tensor, is_sketch

def is_image_a_sketch(img, saturation_threshold=20):
    """
    Auto-detects if an image is a sketch or a real photo.
    Heuristic: Sketches are grayscale or have very low color saturation.
    """
    if len(img.shape) == 2: # It's already grayscale
        return True

    # Convert to HSV color space
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    # Calculate average saturation (S channel)
    avg_saturation = np.mean(hsv[:, :, 1])

    print(f"Average Saturation: {avg_saturation}")
    return avg_saturation < saturation_threshold

def postprocess_image(tensor):
    # De-normalize from [-1, 1] to [0, 255]
    tensor = (tensor * 0.5 + 0.5) * 255
    img = tf.cast(tensor, tf.uint8).numpy()
    return img

# --- Flask Routes ---
@app.route('/')
def index():
    # Render the main HTML page
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict():
    if 'file' not in request.files:
        return jsonify({'error': 'No file part'})

    file = request.files['file']
    if file.filename == '':
        return jsonify({'error': 'No selected file'})

    if file:
        try:
            img_bytes = file.read()

            # 1. Preprocess and Auto-Detect
            input_tensor, is_sketch = preprocess_image(img_bytes)

            # 2. Run inference
            if is_sketch:
                print("Detected sketch, converting to photo...")
                output_tensor = gen_SketchToPhoto(input_tensor, training=False)
            else:
                print("Detected photo, converting to sketch...")
                output_tensor = gen_PhotoToSketch(input_tensor, training=False)

            # 3. Post-process
            output_img = postprocess_image(output_tensor[0]) # Remove batch dim

            # 4. Encode as Base64 to send back to browser
            # Convert RGB (from TF) to BGR (for cv2.imencode)
            output_img_bgr = cv2.cvtColor(output_img, cv2.COLOR_RGB2BGR)
            _, img_encoded = cv2.imencode('.jpg', output_img_bgr)
            img_base64 = base64.b64encode(img_encoded).decode('utf-8')

            return jsonify({'image': f'data:image/jpeg;base64,{img_base64}'})

        except Exception as e:
            print(f"Prediction error: {e}")
            return jsonify({'error': 'Failed to process image.'})

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)

Overwriting app.py


In [14]:
# ---------------------------------
# STEP 1: Mount your Google Drive
# ---------------------------------
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
# ---------------------------------
# STEP 2: Copy the files
# ---------------------------------
print("Copying checkpoints to Google Drive... DO NOT DISCONNECT.")
!cp -r ./checkpoints/train /content/drive/MyDrive/my_cyclegan_checkpoints
print("--- COPY COMPLETE! ---")

Copying checkpoints to Google Drive... DO NOT DISCONNECT.
cp: cannot stat './checkpoints/train': No such file or directory
--- COPY COMPLETE! ---
